In [172]:
import cv2
import numpy as np
import pywt
import numpy as np
from PIL import Image
import pywt
from scipy.linalg import svd, diagsvd, inv
from cryptography.fernet import Fernet
import random

In [208]:
import numpy as np
import cv2
import pywt
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from base64 import urlsafe_b64encode
import os

class SVDWatermarking:
    def __init__(self, host_image_path, watermark_image_path, scaling_factor=0.7, arnold_key=None, key=None):
        self.host_image = cv2.imread(host_image_path, cv2.IMREAD_GRAYSCALE)
        self.watermark_image = cv2.imread(watermark_image_path, cv2.IMREAD_GRAYSCALE)
        self.scaling_factor = scaling_factor
        self.arnold_key = arnold_key or 1
        self.key = key or os.urandom(24)
        self.watermark_dimensions = self.watermark_image.shape
        self.block_size = 8
        self.iv = os.urandom(16)  # Initialization vector for AES


    def arnold_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (x + y) % w
                    new_y = (x + 2 * y) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        print(str(result)+" transform ")
        return result

    def arnold_inverse_transform(self, image, iterations):
        result = np.copy(image)
        h, w = image.shape
        for _ in range(iterations):
            for x in range(w):
                for y in range(h):
                    new_x = (2 * x - y) % w
                    new_y = (y - x) % h
                    result[new_y, new_x] = image[y, x]
            image = np.copy(result)
        print(str(result)+" inverse ")
        return result

    def embed_watermark(self):
        watermark = (self.watermark_image).astype(int)
        watermark = self.arnold_transform(watermark, self.arnold_key)
        blocks = [np.hsplit(row, self.host_image.shape[1] // self.block_size) for row in np.vsplit(self.host_image, self.host_image.shape[0] // self.block_size)]
        blocks = [block.astype(np.float32) for row in blocks for block in row]
        selected_blocks_indices = np.argsort([np.sum(np.abs(block)) for block in blocks])[:np.prod(self.watermark_dimensions)]
    
        selected_blocks = [blocks[idx] for idx in selected_blocks_indices]

        watermarked_image = np.copy(self.host_image)
        encrypted_coords = []
        backend = default_backend()
        key = urlsafe_b64encode(self.key)
        cipher = Cipher(algorithms.AES(key), modes.CBC(self.iv), backend=backend)
        encryptor = cipher.encryptor()
        padder = padding.PKCS7(algorithms.AES.block_size).padder()


        for i, block in enumerate(selected_blocks):
        # Apply DWT and perform SVD
            coeffs = pywt.dwt2(block, 'haar')
            LL, (LH, HL, HH) = coeffs
            U, S, V = np.linalg.svd(LL, full_matrices=False)
            watermark_bit = watermark.flat[i]
            mu = (np.abs(U[2, 1]) + np.abs(U[3, 1])) / 2
            adaptive_strength = mu * 0.1  # Example: Scale changes by 10% of mu 


            if i % 2 == 0:  # Even index, modify U
                U[2, 1] = np.sign(U[2, 1]) * (mu + (self.scaling_factor if watermark_bit else -self.scaling_factor))
                U[3, 1] = np.sign(U[3, 1]) * (mu - (self.scaling_factor if watermark_bit else self.scaling_factor))
            else:  # Odd index, modify V
                V[2, 1] = np.sign(V[2, 1]) * (mu + (self.scaling_factor if watermark_bit else -self.scaling_factor))
                V[3, 1] = np.sign(V[3, 1]) * (mu - (self.scaling_factor if watermark_bit else self.scaling_factor))

            LL_new = np.dot(U, np.dot(np.diag(S), V))
            coeffs_new = (LL_new, (LH, HL, HH))
            block_new = pywt.idwt2(coeffs_new, 'haar')

            row, col = np.unravel_index(selected_blocks_indices[i], (self.host_image.shape[0] // self.block_size, self.host_image.shape[1] // self.block_size))
            watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size] = block_new.astype(np.uint8)

            # Encrypt coordinates, re-initialize padder and encryptor each iteration
            padder = padding.PKCS7(algorithms.AES.block_size).padder()
            encryptor = cipher.encryptor()  # Reinitialize encryptor for each block

            coords = f"{row},{col}".encode()
            padded_data = padder.update(coords) + padder.finalize()
            encrypted_coords.append(encryptor.update(padded_data) + encryptor.finalize())
        return watermarked_image, encrypted_coords

    def extract_watermark(self, watermarked_image, encrypted_coords):
        decrypted_coords = []
        backend = default_backend()
        key = urlsafe_b64encode(self.key)
        cipher = Cipher(algorithms.AES(key), modes.CBC(self.iv), backend=backend)
        decryptor = cipher.decryptor()
        unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
        watermark_bits = []

        for coord in encrypted_coords:
            # Decrypt coordinates, re-initialize unpadder and decryptor each iteration
            unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
            decryptor = cipher.decryptor()  # Reinitialize decryptor for each block

            decrypted_data = decryptor.update(coord) + decryptor.finalize()
            decrypted_coords.append(unpadder.update(decrypted_data) + unpadder.finalize())
            row, col = map(int, decrypted_coords[-1].decode().split(','))

            block = watermarked_image[row * self.block_size:(row + 1) * self.block_size, col * self.block_size:(col + 1) * self.block_size]
            coeffs = pywt.dwt2(block, 'haar')
            LL, _ = coeffs
            U, _, _ = np.linalg.svd(LL, full_matrices=False)

            bit = 1 if U[2, 1] > U[3, 1] else 0
            watermark_bits.append(bit)


        watermark_image = np.array(watermark_bits).reshape(self.watermark_dimensions)
        watermark_image = self.arnold_inverse_transform(watermark_image, self.arnold_key)
        return watermark_image
    def calculate_correlation_coefficient(self, original_watermark, extracted_watermark):
        original_watermark = original_watermark.flatten()
        extracted_watermark = extracted_watermark.flatten()
        return np.corrcoef(original_watermark, extracted_watermark)[0, 1]

    def calculate_bit_error_rate(self, original_watermark, extracted_watermark):

        original_watermark = original_watermark.flatten()
        extracted_watermark = extracted_watermark.flatten()
        num_errors = np.sum(original_watermark != extracted_watermark)
        return num_errors / len(original_watermark)

    def test_watermark(self):


        watermarked_image, encrypted_coords = self.embed_watermark()
        extracted_watermark = self.extract_watermark(watermarked_image, encrypted_coords)

        correlation_coefficient = self.calculate_correlation_coefficient(self.watermark_image, extracted_watermark)
        ber = self.calculate_bit_error_rate(self.watermark_image, extracted_watermark)

        print(f"Correlation Coefficient: {correlation_coefficient:.4f}")
        print(f"Bit Error Rate (BER): {ber:.4f}")




In [209]:
arnold_key = 1 # Key for Arnold transform
watermarker = SVDWatermarking('/Users/kabir/Downloads/Watermark/images/1000.PNG', '/Users/kabir/Downloads/Watermark/images/dice_water.png',arnold_key=arnold_key)
watermarked_image, encrypted_coords = watermarker.embed_watermark()

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]] transform 


In [210]:
watermarker.test_watermark()

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]] transform 
[[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 [1 1 0 ... 0 1 1]] inverse 
Correlation Coefficient: 0.0553
Bit Error Rate (BER): 0.8467


In [195]:
print(encrypted_coords)

[b'>\xf0\x96)\x9a\tIL\x06\nZ\xd6\xccx\xc3Z', b'\xda\xed\xd8\x04?l\x13\xb7\xf5]Z\xc6\xcc\xa84\xf3', b'\\S\xb5Z7z\xcfT\x1a3\xcfk\x96N\x8c\xa2', b'\xc1\xb3\x99\r\x168\x1fTA\x95\x1e\x86"\xf9\xf5\xa1', b'\xcde!6\x18\x92cy}\xed}\x1d\xf8)\xa3X', b'F%y\xf0\x8e\x19\r\xe30\xc0a\xc7#\x88UD', b'\x84\x01"CV\x97\xde+\xb1\xd3\xca\xda\x8dy\x9f9', b'\xb9CM\xb5\x1aZ}\xcan\xb2\xa9\x12\x1f\xa1\x81\xf6', b"2'C@wg\x1c\x18\xbe\x02T\x84\xe6D\re", b'\xab_\x8e\xe4\x06v\xa4Qz\x80UR\xc0\xea.\xce', b'N8\xed\x9cj\xb1\x9a\x19\xaa\xe7\xaeLf\x8d\xf8\x92', b'\xbc\xb2\xf3\x19\xc1\xdb\x044\x0f\xc8\xe7a\xbd\xf3\xdcW', b'Z\xb4p\xd7h\x15\xbfg\x18\xe6\x8aT\xc8Y\xb0\xec', b'\xe7\x9d\x1fb\x98\xb2\xb7J\x82L\x89:\xf7\x0f\xbfN', b'\x04>*\x10\x08h\x9b\xc61\xee\x91\xdd\xa7\x04\x12Z', b'\x11\x95\xd3\xa6D=/e^\xa6\x90|\x1e,\xbd\x90', b'm\xefC\xa1H\xff\xf4]\xa2\xb6\xf0\x08+F\xf8k', b'\x1e{!\x99\xd7|L5\xb4\xe4\x0f\xe4\x986\x11\xf0', b"\xf6\x04\x06\xca\x94\xb8\x8a\xda\xb5\x19\xdc\xc71\xd9'\xc6", b'hz\x88\xfb\xcd\x19\xa6g\x7fB\xabJ+D\x7f\

In [176]:
image = Image.fromarray(watermarked_image)
image.show()

In [177]:
extracted_watermark = watermarker.extract_watermark(watermarked_image,encrypted_coords)


[[0 0 0 ... 1 1 1]
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 [1 1 0 ... 0 1 1]] inverse 


In [178]:
extracted_watermark

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 1, 0, ..., 0, 1, 1]])

In [179]:
# Ensure that the values are within the 0-255 range for uint8
normalized_watermark = np.clip(extracted_watermark * 255, 0, 255)

# Convert the data type to uint8
watermark_uint8 = normalized_watermark.astype(np.uint8)

# Create the image
image = Image.fromarray(watermark_uint8)
image.show()